In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import analysis_parameters as ap

chunks = {"Xdim":1024, "time":30}

In [2]:
pthick = xr.open_dataset(ap.get_file("GEOS","TWP_3D","pthick"), chunks=chunks).pthick
pthick

<xarray.DataArray 'pthick' (time: 720, lev: 181, Xdim: 97152)>
dask.array<open_dataset-ca151567f87eba72e76fe8ecccb86a1bpthick, shape=(720, 181, 97152), dtype=float32, chunksize=(720, 181, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-30 ... 2020-02-28T23:00:00
    lons     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
    lats     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 177.0 178.0 179.0 180.0 181.0
Dimensions without coordinates: Xdim
Attributes:
    standard_name:  air_pressure_thickness
    long_name:      pressure_thickness
    units:          Pa
    CDI_grid_type:  unstructured
    origin:         DELP

In [3]:
pres_sfc = xr.open_dataset(ap.get_file("GEOS","TWP","ps"), chunks=chunks).ps.sel(time=pthick.time)
pres_sfc

<xarray.DataArray 'ps' (time: 720, Xdim: 97152)>
dask.array<getitem, shape=(720, 97152), dtype=float32, chunksize=(720, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-30 ... 2020-02-28T23:00:00
    lons     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
    lats     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
Dimensions without coordinates: Xdim
Attributes:
    standard_name:  surface_air_pressure
    long_name:      surface_pressure
    units:          Pa
    CDI_grid_type:  unstructured
    origin:         PS

In [4]:
pres_sfc.shape, pthick.shape

((720, 97152), (720, 181, 97152))

In [5]:
pthick_cumsum_neg = (-1*pthick).cumsum(axis=1)
pthick_cumsum_neg
# pthick_cumsum_neg.mean(axis=(0,1)).plot()

<xarray.DataArray 'pthick' (time: 720, lev: 181, Xdim: 97152)>
dask.array<nancumsum, shape=(720, 181, 97152), dtype=float32, chunksize=(720, 181, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-30 ... 2020-02-28T23:00:00
    lons     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
    lats     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 177.0 178.0 179.0 180.0 181.0
Dimensions without coordinates: Xdim

In [6]:
pthick_cumsum_neg.to_netcdf("/scratch/b/b380883/pthick_cumsum_neg_geos.nc")

In [6]:
ps_expand = pres_sfc.expand_dims(dim={"lev":181}, axis=1).assign_coords({"lev":pthick.lev})
print(ps_expand.shape, pthick_cumsum_neg.shape)
ps_expand

(720, 181, 97152) (720, 181, 97152)


<xarray.DataArray 'ps' (time: 720, lev: 181, Xdim: 97152)>
dask.array<transpose, shape=(720, 181, 97152), dtype=float32, chunksize=(720, 181, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-30 ... 2020-02-28T23:00:00
    lons     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
    lats     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 177.0 178.0 179.0 180.0 181.0
Dimensions without coordinates: Xdim
Attributes:
    standard_name:  surface_air_pressure
    long_name:      surface_pressure
    units:          Pa
    CDI_grid_type:  unstructured
    origin:         PS

In [ ]:
ps_expand.to_netcdf("/scratch/b/b380883/pres_sfc_expanded_geos.nc")

In [ ]:
del pthick_cumsum_negcumsum_neg, ps_expand

In [2]:
ps_expand = xr.open_dataarray("/scratch/b/b380883/pres_sfc_expanded_geos.nc", chunks=chunks)
pthick_cumsum_neg = xr.open_dataarray("/scratch/b/b380883/pthick_cumsum_neg_geos.nc", chunks=chunks)

In [3]:
ps_expand

<xarray.DataArray 'ps' (time: 720, lev: 181, Xdim: 97152)>
dask.array<open_dataset-344aab772f6f50f8c57ea2fc381ebcb6ps, shape=(720, 181, 97152), dtype=float32, chunksize=(30, 181, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-30 ... 2020-02-28T23:00:00
    lons     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
    lats     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 177.0 178.0 179.0 180.0 181.0
Dimensions without coordinates: Xdim
Attributes:
    standard_name:  surface_air_pressure
    long_name:      surface_pressure
    units:          Pa
    CDI_grid_type:  unstructured
    origin:         PS

In [4]:
pthick_cumsum_neg


<xarray.DataArray 'pthick' (time: 720, lev: 181, Xdim: 97152)>
dask.array<open_dataset-cb20e0c1905d4b25cff4c79c61151badpthick, shape=(720, 181, 97152), dtype=float32, chunksize=(30, 181, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-30 ... 2020-02-28T23:00:00
    lons     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
    lats     (Xdim) float32 dask.array<chunksize=(1024,), meta=np.ndarray>
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 177.0 178.0 179.0 180.0 181.0
Dimensions without coordinates: Xdim

In [5]:
pa = ps_expand + pthick_cumsum_neg
pa

<xarray.DataArray (time: 720, lev: 181, Xdim: 97152)>
dask.array<add, shape=(720, 181, 97152), dtype=float32, chunksize=(30, 181, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-30 ... 2020-02-28T23:00:00
    lons     (Xdim) float32 143.0 143.0 143.0 143.0 ... 153.0 153.0 153.0 153.0
    lats     (Xdim) float32 4.98 4.949 4.918 4.887 ... -4.912 -4.945 -4.978
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 177.0 178.0 179.0 180.0 181.0
Dimensions without coordinates: Xdim

In [ ]:
pa = pa.compute()

In [ ]:
pa

In [6]:
pa = pa.assign_attrs({"standard_name":"air_pressure",
                          "long_name":"pressure_of_air_at_levs",
                          "method":"pthick cumulatively subtracted from pressure at the surface",
                          "unit":"Pa"})
pa

<xarray.DataArray (time: 720, lev: 181, Xdim: 97152)>
dask.array<add, shape=(720, 181, 97152), dtype=float32, chunksize=(30, 181, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-30 ... 2020-02-28T23:00:00
    lons     (Xdim) float32 143.0 143.0 143.0 143.0 ... 153.0 153.0 153.0 153.0
    lats     (Xdim) float32 4.98 4.949 4.918 4.887 ... -4.912 -4.945 -4.978
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 177.0 178.0 179.0 180.0 181.0
Dimensions without coordinates: Xdim
Attributes:
    standard_name:  air_pressure
    long_name:      pressure_of_air_at_levs
    method:         pthick cumulatively subtracted from pressure at the surface
    unit:           Pa

In [7]:
pa = xr.Dataset({"pa":pa}, attrs=pa.attrs)
pa

<xarray.Dataset>
Dimensions:  (time: 720, Xdim: 97152, lev: 181)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-30 ... 2020-02-28T23:00:00
    lons     (Xdim) float32 143.0 143.0 143.0 143.0 ... 153.0 153.0 153.0 153.0
    lats     (Xdim) float32 4.98 4.949 4.918 4.887 ... -4.912 -4.945 -4.978
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 177.0 178.0 179.0 180.0 181.0
Dimensions without coordinates: Xdim
Data variables:
    pa       (time, lev, Xdim) float32 dask.array<chunksize=(30, 181, 1024), meta=np.ndarray>
Attributes:
    standard_name:  air_pressure
    long_name:      pressure_of_air_at_levs
    method:         pthick cumulatively subtracted from pressure at the surface
    unit:           Pa

In [ ]:
pa.to_netcdf("/work/bb1153/b380883/TWP/TWP_3D_GEOS_pa_20200130-20200228.nc")